In [6]:
from konlpy.tag import Mecab
try:
    import ujson
except ImportError:
    import json

In [7]:
mecab = Mecab()

In [8]:
with open('./data/2017_03.json') as f:
    data = json.load(f)

In [9]:
data[0]

{'date': '2017-03-01',
 'text': '플레이어 이동시키기 플레이어 닫기\n\n<앵커>\n\n\n\n98년 전 3월 1일 태극기를 들고 거리로 나서서 독립을 외치던 우리의 할아버지, 할머니들이 오늘(1일) 서울 한복판이 이렇게 둘로 갈라진 모습을 보면 뭐라고 생각하실까요? 탄핵 찬성과 반대, 각각 목소리를 높인 집회 참가자들 머리 위로는 비가 내렸고, 날씨는 을씨년스러웠습니다.\n\n\n\n먼저 권란 기자가 오늘 집회 상황 종합했습니다.\n\n\n\n<기자>\n\n\n\n광화문 광장과 맞닿은 세종대로 남쪽에서 먼저 탄핵 반대 집회가 시작됐습니다.\n\n\n\n서울광장에 이어 숭례문까지 태극기를 든 참가자들로 꽉 찼습니다.\n\n\n\n대형 태극기에다 성조기까지 등장했고, 고 박정희 전 대통령의 얼굴이 그려진 깃발도 배포됐습니다.\n\n\n\n참가자들은 헌재의 탄핵 결정을 앞두고 목청 높여 탄핵에 반대했습니다.\n\n\n\n \n\n\n\n집회 시작 네 시간쯤 뒤부터는 탄핵 반대 집회 측으로선 처음으로 청와대 방향 행진을 시작했습니다.\n\n\n\n헌재와 도심 등 5개 경로로 행진을 이어갔습니다.\n\n\n\n탄핵 반대 집회가 끝나갈 무렵인 오후 5시쯤부터는 탄핵 찬성 집회가 세종대로 북쪽 광화문광장에서 열렸습니다.\n\n\n\n충돌을 우려해 경찰이 차벽으로 양측을 막아선 가운데, 촛불을 든 시민들은 헌재의 탄핵 인용 결정을 촉구하며 목소리를 높였습니다.\n\n\n\n \n\n\n\n특히 3.1절을 맞아 위안부 피해자 이용수 할머니가 무대에 올라 아리랑을 부르자, 참가자 모두 따라 부르기도 했습니다.\n\n\n\n \n\n\n\n박근혜 대통령 탄핵 여부가 결정되는 운명의 3월 첫날에 서울 도심 광장을 둘로 나눈 대규모 집회여서 충돌이 걱정됐지만 다행히 아직 불상사는 없는 상황입니다.\n\n\n\n(영상취재 : 신진수·김성일·박진호, 영상편집 : 윤선영)',
 'title': '대통령 운명 결정할 3월…탄핵 찬반에 둘로 나뉜 3·1절'}

In [11]:
tags = set(['NNG', 'NNP', 'VA', 'VV'])
article_list = []
for i, article in enumerate(data):
    if i % 1000 == 0:
        print(i)
    
    text = article['text'].strip()
    if len(text) > 10:
        article_list.append([w for w, t in mecab.pos(text) if t in tags and len(w) > 1])

        if len(article_list) == 10000:
            break

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [12]:
from collections import Counter

word_count = Counter()

for words_list in article_list:
    word_count.update(words_list)
    
voca = [w for w, freq in word_count.most_common(10000)]
word2id = {w: i for i, w in enumerate(voca)}

article_list_ = []
for words_list in article_list:
    words = [w for w in words_list if w in word2id]
    if len(words) >= 5:
        article_list_.append(words)

In [13]:
import numpy as np
from sklearn.preprocessing import normalize

tdm = np.zeros((len(article_list_), len(voca)), dtype=np.float32)
for i, words in enumerate(article_list_):
    for word in words:
        try:
            tdm[i, word2id[word]] += 1
        except KeyError:
            pass
        
normalize(tdm, copy=False)

array([[0.47434166, 0.10540926, 0.42163703, ..., 0.        , 0.        ,
        0.        ],
       [0.34174308, 0.42717883, 0.17087154, ..., 0.        , 0.        ,
        0.        ],
       [0.39361864, 0.45576894, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.04199605, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03155972, 0.06311944, 0.03155972, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [14]:
from sklearn.decomposition import NMF

K = 10
nmf = NMF(n_components=K)
W = nmf.fit_transform(tdm)
H = nmf.components_

In [15]:
for k in range(K):
    print(f"{k}th topic")
    for i in H[k].argsort()[::-1][:20]:
        print(voca[i], end=' ')
    print()

0th topic
탄핵 기각 국민 운동 총궐기 본부 반대 대통령 집회 주최 절인 기국 서울 사진 열리 동취 오후 재단 사거리 단체 
1th topic
방송 사고 등장 부부 행복 스타 웃음 사탕 켈리 지사 출연 안희정 세월 차량 사진 김문수 중대 인단 발언 설명 
2th topic
태극기 집회 탄핵 반대 참가자 행진 오후 참석 경찰 단체 의원 촛불 기자 사거리 대한문 이날 기국 보수 흔들 주장 
3th topic
대표 지사 민주당 대선 국민 의원 후보 정치 정당 시장 문재인 더불 주자 탄핵 안희정 자유 국당 기자 이날 강조 
4th topic
대선 사람 지원 불출마 보통 한국 정부 기업 검찰 황교안 인상 금리 주택 소환 청춘 선언 이상 시간 출시 진행 
5th topic
집회 촛불 박근혜 광장 퇴진 탄핵 광화문 행동 촉구 오후 국민 시민 인용 구속 행진 정권 이날 황교안 참가자 만세 
6th topic
서울 기자 뉴스 오후 광장 뉴시스 광화문 참가자 박근혜 종로구 절인 외치 촉구 시장 구호 경찰 전신 무단 참석자 서울시 
7th topic
특검 수사 검찰 국회 연장 국민 특별 검사 최순 대행 박영수 결과 처리 수석 의장 조사 권한 직권상정 국정 개정안 
8th topic
대통령 박근혜 탄핵 주장 정치 사람 청와대 변호사 최순 인단 대리 출마 국정 촉구 심판 문제 국가 대선 발언 소추 
9th topic
탄핵 헌재 심판 헌법 재판관 선고 재판소 결정 대행 권한 국회 인용 변론 평의 이정미 최종 기자 의견 소장 절차 


In [16]:
mecab.pos("최순실")

[('최순', 'NNP'), ('실', 'NNG')]

In [18]:
mecab.pos("삼일절인")

[('삼', 'NR'), ('일', 'NNBC'), ('절인', 'NNG')]